## NOTES

#### RESOURCES

In [ ]:
https://brennan.io/2016/03/02/logging-in-with-requests/
https://kazuar.github.io/scraping-tutorial/

In [2]:
import json
import pandas as pd
import csv
import datetime
import collections
import re
import copy

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

pprint('all done')

'all done'


#### COOKIES & LOGIN

In [ ]:
NOT NECCESSARY B/C THE FORUM LIST DOESN'T REQIURE LOGIN TO ACCESS
However, see Prison Talk Scraper Login Notebook for process.

In [ ]:
domain = 'www.prisontalk.com'

headers =   {
                'Host': domain,
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:44.0) Gecko/20100101 Firefox/44.0',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive'
            }

In [ ]:
username = 'YOUR_USERNAME'
password = 'YOUR_PASSWORD'

login_form = {
                'vb_login_username': username, 
                'vb_login_password': password,
                'do': 'login',
                'url': '/forums/memberlist.php', 
                'vb_login_md5password': '',
                'vb_login_md5password_utf': '',
                's': '',
                'securitytoken': 'guest'
             }

# login_form

In [ ]:
prison_talk = BeautifulSoup(response.content, 'lxml')

In [ ]:
login_url = 'http://www.prisontalk.com/forums/login.php?do=login'

s.post(login_url, data=form)

s = requests.session()
response = s.get(login_url, headers=headers)
print response.status_code

In [ ]:
login_form = get_login_form(login_page, username, password)
login_form
logged_in = s.post(login_url, data=login_form, headers=headers)
logged_in.status_code

In [ ]:
success_page = BeautifulSoup(logged_in.content)
bool(success_page.find_all(string=re.compile('Redirecting')))

#### REQUEST WEBPAGE AND SAVE TO FILE

In [18]:
forum = 38
order = 'DESC'
sort = 'posts'
pages_per = '100'
page = 1

forum_url = 'http://prisontalk.com/forums/forumdisplay.php?'+ \
                                            'f='+str(forum)+ \
                                            '&order='+order+ \
                                            '&sort='+sort+ \
                                            '&pp='+pages_per+ \
                                            '&page='+str(page)
forum_url

'http://prisontalk.com/forums/forumdisplay.php?f=38&order=DESC&sort=posts&pp=100&page=1'

In [20]:
s = requests.session()
response = s.get(forum_url)

print response.status_code
    
if response.status_code == 200:

    thread_list = BeautifulSoup(response.content, 'lxml')
    
    with open("examples/thread_list.html", "w") as file:
        file.write(str(thread_list))

200


#### LOAD WEBPAGE FROM FILE & REMOVE STRING LITERALS (THEY'RE THEY WORST)

In [132]:
posts_no_nav_w_thanks = 'posts_no_nav_w_thanks'
posts_w_nav_w_thanks = 'posts_w_nav_w_thanks'
posts_w_nav = 'posts_w_nav'
posts_w_poll_nav = 'posts_w_poll_nav'


html_page = open('examples/' + posts_w_nav_w_thanks + '.html', 'r').read()
posts_page = BeautifulSoup(html_page, 'html.parser')
posts_page = str(posts_page)
no_literals = posts_page.replace('\n', '').replace('\t', '').replace('\r', '')


posts_page = BeautifulSoup(no_literals, 'html.parser')#skip u'html PUBLIC "-//W3C//

In [17]:
type(posts_page)

bs4.BeautifulSoup

#### EXTRACT NAV FOR NUMBER OF PAGES

In [18]:
# NOTE: THIS DOES NOT ASSUME EVERY THREAD PAGE WILL HAVE >100 THREADS AND THEREFORE A NAV

def get_num_post_pages(el):
    
    try:
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = el.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
        
    return num_pages

get_num_post_pages(posts_page)  #263

u'8'

In [220]:
def get_thanks(page_of_posts):
    
    thankers = []

    # <div id="post_thanks_box_4543095"><div...Say Thank You to...div> <a href="member.php?u=166646"...">Catoneswife</a> (05-13-2009)
    thanks_boxes = get_thanks_boxes(page_of_posts)
    
    for thanks_box in thanks_boxes:
        
        # u'post_thanks_box_4543095'
        thanked_post_id = thanks_box['id'].split('_')[-1]
        
        links_of_thanks = thanks_box.find_all('a')
#         dates_of_thanks = thanks_box.find_all('a')
            
        for link_of_thanks in links_of_thanks:  
        # <a href="member.php?u=166646" rel="nofollow">Catoneswife</a>(02-27-2015)</div>
        
            thanker = {}
        
            thanker['thanked_post_id'] = thanked_post_id
            
            # <a>...</a>(02-27-2015),</div>        
            date_of_thanks = link_of_thanks.next_sibling
            # u'\xa0(05-13-2009),
            date_of_thanks = date_of_thanks.encode('ascii', 'ignore').decode('ascii')
            # (05-13-2009),
            thanker['date_of_thanks'] = re.sub('[(),]', '', date_of_thanks)

#             u'member.php?u=166646'
            thanker['thanker_user_id'] = link_of_thanks['href'].split('=')[-1]

            # u'Catoneswife'
            thanker['thanker_user_name'] = link_of_thanks.text

            thankers.append(thanker)

    return thankers

# NOTE: THANKER USER ID AND POST ID SHOULD NEVER APPEAR TWICE AS
# THAT WOULD INDICATE THAT THE SAME PERSON THANKED THE POST TWICE WHICH
# TO MY KNOWLEDGE IS NOT POSSIBLE

# [{'date_of_thanks': u'05-13-2009 ','thanked_post_id': u'4543095',
#   'thanker_user_id': u'166646','thanker_user_name': u'Catoneswife'},
#  {'date_of_thanks': u'02-27-2015','thanked_post_id': u'4543095',
#   'thanker_user_id': u'377524','thanker_user_name': u'Maushel'}

thanks = get_thanks(posts_page)
# thanks

In [221]:
def get_thanks_boxes(el):
    
    thanks_boxes = el.find_all(id=re.compile('post_thanks_box'))
    
    return thanks_boxes
    
# thanks_boxes = get_thanks_boxes(posts_page)
# print thanks_boxes[0].prettify()

#### WRITE TO CSV (IN THE FUTURE CHANGE TO DICTWRITER)

In [385]:
fieldnames = ['number', 'name', 'threads', 'posts', 'description']

with open('data/forum_list.csv', 'w') as csv_file:
    
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(fieldnames)
    
    for forum in forums:
        writer.writerow(forum)